# Problem Statement

Find the value of $d < 1000$ for which $1/d$ contains the longest recurring cycle in its decimal fraction part.

## Solution 1

We do the obvious brute force solution. For each $d \in [2, 1000]$ compute $1 / d$ using extra precision. Convert the result to a string, and count the length of the longest cycle in the string. Figuring out the length of the longest cycle is a *little* tricky. Code first:

In [ ]:
# %load -s get_cycle_len p26_reciprocal_cycles.py
def get_cycle_len(n, max_so_far = 7):
  # Longest repeating sequence for a number is n - 1 so we'll need a lot of
  # precision for large numbers.
  decimal.getcontext().prec = 2 * n
  # This is just a really complicated way of stripping the leading 0's
  s = str(decimal.Decimal(str(decimal.Decimal(1) / decimal.Decimal(n))[2:]))
  # This is *almost* like longest repeated substring, but we can make it
  # faster because we know the repeated part of the sequence is adjacent.
  # Additional, we *only* care if the repeated part is longer than what
  # we've found so far.
  cycle_len = max_so_far + 1
  for cycle_len in range(max_so_far + 1, n):
    for i in range(len(s) - cycle_len * 2):
      if s[i:i+cycle_len] == s[i+cycle_len:i+2*cycle_len]:
        # Double check that this can't be broken apart into a smaller sequence.
        for j in range(1, (cycle_len // 2) + 1):
          if s[i:i+j] == s[i+j:i+2*j]:
            return 0
        return cycle_len
  return 0


This is the main interesting logic of the brute force function. We convert the decimal result to a string with sufficient precision. Then, we just take slices of the decimal string until we get a matching slice. After we find a matching slice we double check that it's not a result of smaller slices. The digit string for $1/3$ would be "333333333...." so if we took s[0:100] == s[100:200] it would be true! But this is a repeating sequence of length 1, not length 100.

Only thing missing is a driver function to run it:

In [ ]:
# %load -s get_max_cycle_brute p26_reciprocal_cycles.py
def get_max_cycle_brute(n):
  # Brute force that cycle baby.
  num = 0
  max_cycle = 1
  # Skip the boring numbers.
  for i in range(7, n):
    cycle_len = get_cycle_len(i, max_so_far=max_cycle)
    if cycle_len > max_cycle:
      max_cycle = cycle_len
      num = i
  return (num, max_cycle)


## Solution 2

A few observations:

$$
\begin{align}
  \frac{3}{9} &= .333333333 \\
  \frac{34}{99} &= .3434343434 \\
  \frac{345}{999} &= .345345345 \\
\end{align}
$$

Repeated 9's seem to have a very interesting property: We can get a repeated sequence of digits $n$ from a number $n$ via dividing it by an appropriate number of nines.

Also 10's don't have an effect on the repeated sequence. The length of the repeated decimal part of $\frac{1}{3} = \frac{1}{30} = \frac{1}{300} = \cdots $. The prime divisors of 10 are 2 & 5, and interestingly it seems like 2 & 5 also have some interesting properties:

$$
\begin{align}
  \frac{1}{11} &= .0909090 \\
  \frac{1}{22} &= .045454545 \\
  \frac{1}{44} &= .0227272727 \\
  \frac{1}{88} &= .0113535353 \\
  \cdots \\
  \frac{1}{55} &= .0181818 \\
  \frac{1}{275} &= .00363636 \\
\end{align}
$$

For some reason, multiples of 2 & 5 don't affect the **length** of the repeating digits. This was pretty non obvious to me at first, but we can use our first observation to show why. Let's take $\frac{1}{11}$ as an example again, but let's rewrite it as $\frac{9}{99}$. It's easy to see from this to see that the repeating decimal is 909090...  We have a 9 on top and 2 nine's on the bottom. So let's add a factor of 5 to our starting number to 11:

$$
\begin{align}
  \frac{1}{55} &= \frac{1}{11} \cdot \frac{1}{5} \\
  &= \frac{9}{99} \cdot \frac{1}{5} \\
  &= \frac{90}{990} \cdot \frac{1}{5} \\
  & = \frac{18}{990}
\end{align}
$$

Remember that 10's don't have an effect on the repeated sequence. So, the repeated sequence will just be 181818. So even though the digits changed, the denominator still only has **two** 9's in it, Giving us a repeated sequence that is still only two digits long. We can argue similarly for any generic $\frac{a}{b}$:

$$
\frac{a}{b} \cdot \frac{1}{5} = \frac{a}{b} \cdot \frac{2}{10} = \frac{2a}{10b}
$$

Since the number of 9's won't change in the denominator, the number of repeated digits won't change. The proof for two is the same:

$$
\frac{a}{b} \cdot \frac{1}{2} = \frac{a}{b} \cdot \frac{5}{10} = \frac{5a}{10b}
$$

One final observation and then to the code. Again any repeated decimal can be expressed as a fraction over some number of 9's. So we have $\frac{1}{d} = \frac{n}{i}$ where $d$ is some integer, $n$ is some integer, and $i$ is some number of 9's. By multiplying through the denominators, we get: $i = n \cdot d$. In otherwords, $n$ and $d$ are both factors of $i$. Kind of an interesting fact: For all numbers that aren't divisble by 2 or 5, there exists some number constructed of repeated 9's that it is a factor of. So, all we need to do is to find the length of our number $i$, since that determines how long the repeating cycle is. Here's the code:

In [ ]:
# %load -s get_max_cycle_fast p26_reciprocal_cycles.py
def get_max_cycle_fast(n):
  num = 0
  max_cycle = 1
  # Skip the boring numbers.
  for i in range(7, n):
    # If a number n is divisble by 2 or 5, it's repeating decimal portion has
    # an equivalent length of n / 2 OR n / 5. So, since we're going from bottom
    # to top, we've already calculated it, and we can skip it entirely.
    if i % 2 == 0 or i % 5 == 0:
      continue
    for j in range(1, i):
      if int("9" * j) % i == 0:
        if j > max_cycle:
          max_cycle = j
          num = i
        break
  return num, max_cycle


In [5]:
import p26_reciprocal_cycles

print(p26_reciprocal_cycles.get_max_cycle_fast(1000))

(983, 982)
